# Fitting a Carbonate Correction for MP-sourced Gibbs free energy of formation data (300 K)

Author: Matthew McDermott

Date: May 24, 2023

#### Imports

In [1]:
import numpy as np
import pandas
import plotly.express as px
import plotly.graph_objects as go
from mp_api.client import MPRester
from pymatgen.core.composition import Composition, Element
from scipy.optimize import minimize

from rxn_network.entries.entry_set import GibbsEntrySet

%load_ext autoreload
%autoreload 2

### Chemical systems

In [2]:
metals = ["Ag", "Ba", "Ca", "Cd", "Cs", "Fe", "K",
          "Li", "Mg", "Mn", "Na", "Pb", "Rb", "Sr", "Zn"]  # all metals with carbonate energies available

systems = [(m, "C", "O") for m in metals]

### Downloading Materials Project (MP) data using API

In [3]:
all_entries = {}

for system in systems:
    with MPRester() as mpr:
        entries = mpr.get_entries_in_chemsys(system)

    all_entries[system[0]] = entries

Retrieving ThermoDoc documents:   0%|          | 0/140 [00:00<?, ?it/s]

Retrieving ThermoDoc documents:   0%|          | 0/173 [00:00<?, ?it/s]

Retrieving ThermoDoc documents:   0%|          | 0/175 [00:00<?, ?it/s]

Retrieving ThermoDoc documents:   0%|          | 0/125 [00:00<?, ?it/s]

Retrieving ThermoDoc documents:   0%|          | 0/148 [00:00<?, ?it/s]

...

Retrieving ThermoDoc documents:   0%|          | 0/178 [00:00<?, ?it/s]

Retrieving ThermoDoc documents:   0%|          | 0/146 [00:00<?, ?it/s]

Retrieving ThermoDoc documents:   0%|          | 0/173 [00:00<?, ?it/s]

Retrieving ThermoDoc documents:   0%|          | 0/150 [00:00<?, ?it/s]

Retrieving ThermoDoc documents:   0%|          | 0/138 [00:00<?, ?it/s]

### Acquiring GibbsComputedEntries and corresponding experimental entries

In [20]:
mp_entries = []
exp_entries = []

temp = 300  # Kelvin

for m, entries in all_entries.items():
    gibbs = GibbsEntrySet.from_entries(entries, temp, include_nist_data=False, apply_carbonate_correction=False)
    gibbs_exp = GibbsEntrySet.from_entries(entries, temp, include_nist_data=True, include_freed_data=True, apply_carbonate_correction=False)

    oxidation_states = Element(m).common_oxidation_states

    for os in oxidation_states:
        formula = Composition(f"{m}2(CO3){os}").reduced_formula

        mp_entry = None
        exp_entry = None

        try:
            mp_entry = gibbs.get_min_entry_by_formula(formula)
            exp_entry = gibbs_exp.get_min_entry_by_formula(formula)
        except:
            continue

        if exp_entry.is_experimental:
            mp_entries.append(mp_entry)
            exp_entries.append(exp_entry)

/Users/mcdermott/PycharmProjects/reaction-network/src/rxn_network/entries/entry_set.py:555: FutureWarning:

This method is deprecated. Use from_computed_entries instead.



In [21]:
energies_calc = []
energies_exp = []
names = []

for e_exp, e_calc in zip(exp_entries, mp_entries):
    formula = e_exp.composition.reduced_formula
    num_atoms_per_fu = Composition(formula).num_atoms

    energy_exp = e_exp.energy_per_atom * num_atoms_per_fu
    energy_calc = e_calc.energy_per_atom * num_atoms_per_fu
    name = e_calc.composition.reduced_formula

    energies_exp.append(energy_exp)
    energies_calc.append(energy_calc)
    names.append(name)

### Plot before correction

In [107]:
df = pandas.DataFrame({"x":energies_exp, "y":energies_calc, "name":names})

scatter = px.scatter(df, x="x", y="y", hover_name="name", symbol="name", color="name", color_discrete_sequence=[px.colors.qualitative.Dark24[i] for i in range(len(df_new))])
line = px.line(x=[-12, -4], y=[-12, -4])

fig = go.Figure(scatter.data + line.data)
fig.update_layout({"title": "Carbonate formation energies",
                   "xaxis_title":"Experimental dGf (eV/f.u.)",
                   "yaxis_title":"Calculated dGf (eV/f.u.)"})
fig.show()

### Fitting correction with scipy.optimize.minimize
Minimizing the mean absolute error (MAE) for all corrected phases.

In [108]:
def func(correction):
    return np.mean(np.abs((df["y"] + correction) - df["x"]))  # definition of MAE

In [109]:
final_correction = minimize(func, 0.25).x[0]

In [110]:
len(df_new)

15

In [111]:
df_new = df.copy()
df_new["y"] = df_new["y"] + final_correction

scatter_new = px.scatter(df_new, x="x", y="y", hover_name="name",
                         color_discrete_sequence=[px.colors.qualitative.Dark24[i] for i in range(len(df_new))],
                         symbol="name", color="name")
line_new = px.line(x=[-12, -4], y=[-12, -4])

fig_new = go.Figure(scatter_new.data + line_new.data)
fig_new.update_layout({"title": "Corrected carbonate formation energies",
                   "xaxis_title":"Experimental dGf (eV/f.u.)",
                   "yaxis_title":"Calculated dGf (eV/f.u.)"})
fig_new.show()

## Final correction value (eV per CO$_3^{2-}$):

In [112]:
print(final_correction)

0.8301272393864071


## Figure

In [119]:
from copy import deepcopy

import plotly.graph_objects as go
from plotly.subplots import make_subplots

combo_fig = make_subplots(rows=2, cols=1, shared_xaxes=True)

for t in fig.data:
    t = deepcopy(t)
    t["marker"]["size"] = 12
    combo_fig.add_trace(
        t,
        row=1, col=1
    )

for t in fig_new.data:
    t = deepcopy(t)
    t["showlegend"] = False
    t["marker"]["size"] = 12
    combo_fig.add_trace(
        t,
        row=2, col=1
    )

combo_fig.update_layout({"width":900, "height":800,
                         "xaxis1":{"tickfont":{"size":16}},
                         "xaxis2":{"tickfont":{"size":16}, "title": "Experimental formation energy (eV/atom)", "titlefont":{"size":20}},
                         "yaxis1":{"tickfont":{"size":16}, "title": "Raw formation <br> energy (eV/atom)", "titlefont":{"size":20}},
                         "yaxis2":{"tickfont":{"size":16}, "title": "Corrected formation <br> energy (eV/atom)", "titlefont":{"size":20}},
                         "margin":{"t":20}})

combo_fig.add_annotation(x=-0.15, y=1.02, xref="paper", yref="paper", text="<b>a</b>",
                   showarrow=False, font={"family":"Helvetica","size":28})
combo_fig.add_annotation(x=-0.15, y=0.44, xref="paper", yref="paper", text="<b>b</b>",
                   showarrow=False, font={"family":"Helvetica","size":28})

combo_fig.write_image("carbonate_correction.png", scale=3)
combo_fig

In [32]:
fig.data

(Scatter({
     'hovertemplate': '<b>%{hovertext}</b><br><br>name=Ag2CO3<br>x=%{x}<br>y=%{y}<extra></extra>',
     'hovertext': array(['Ag2CO3'], dtype=object),
     'legendgroup': 'Ag2CO3',
     'marker': {'color': '#1F77B4', 'symbol': 'circle'},
     'mode': 'markers',
     'name': 'Ag2CO3',
     'orientation': 'v',
     'showlegend': True,
     'x': array([-4.52349074]),
     'xaxis': 'x',
     'y': array([-5.71636143]),
     'yaxis': 'y'
 }),
 Scatter({
     'hovertemplate': '<b>%{hovertext}</b><br><br>name=BaCO3<br>x=%{x}<br>y=%{y}<extra></extra>',
     'hovertext': array(['BaCO3'], dtype=object),
     'legendgroup': 'BaCO3',
     'marker': {'color': '#FF7F0E', 'symbol': 'diamond'},
     'mode': 'markers',
     'name': 'BaCO3',
     'orientation': 'v',
     'showlegend': True,
     'x': array([-11.72931073]),
     'xaxis': 'x',
     'y': array([-12.62042317]),
     'yaxis': 'y'
 }),
 Scatter({
     'hovertemplate': '<b>%{hovertext}</b><br><br>name=CaCO3<br>x=%{x}<br>y=%{y}<extra><

In [19]:
fig_new

### Test carbonate correction class

In [11]:
li_entries = GibbsEntrySet.from_entries(all_entries["Li"], 300, include_nist_data=False)

/Users/mcdermott/PycharmProjects/reaction-network/src/rxn_network/entries/entry_set.py:555: FutureWarning:

This method is deprecated. Use from_computed_entries instead.



In [12]:
li2co3 = li_entries.get_min_entry_by_formula("Li2CO3")
li2co3.energy_adjustments

[CarbonateCorrection:
   Name: Carbonate Correction
   Value: 1.656 eV
   Uncertainty: nan eV
   Description: Correction for dGf with (CO3)2- anion, as fit to MP data (300 K). (0.828 eV/atom x 2.0 atoms)
   Generated by: None,
 ConstantEnergyAdjustment:
   Name: Gibbs SISSO Correction
   Value: 1.868 eV
   Uncertainty: 0.600 eV
   Description: Gibbs correction: dGf(300 K) - dHf (298 K) (1.868 eV)
   Generated by: None]

In [13]:
mp_entries = []
exp_entries = []

temp = 300  # Kelvin

for m, entries in all_entries.items():
    gibbs = GibbsEntrySet.from_entries(entries, temp, include_nist_data=False, apply_carbonate_correction=True)
    gibbs_exp = GibbsEntrySet.from_entries(entries, temp, include_nist_data=True, include_freed_data=True, apply_carbonate_correction=False)

    oxidation_states = Element(m).common_oxidation_states

    for os in oxidation_states:
        formula = Composition(f"{m}2(CO3){os}").reduced_formula

        mp_entry = None
        exp_entry = None

        try:
            mp_entry = gibbs.get_min_entry_by_formula(formula)
            exp_entry = gibbs_exp.get_min_entry_by_formula(formula)
        except:
            continue

        if exp_entry.is_experimental:
            mp_entries.append(mp_entry)
            exp_entries.append(exp_entry)

In [14]:
energies_calc = []
energies_exp = []
names = []

for e_exp, e_calc in zip(exp_entries, mp_entries):
    energy_exp = e_exp.energy_per_atom
    energy_calc = e_calc.energy_per_atom
    name = e_calc.composition.reduced_formula

    energies_exp.append(energy_exp)
    energies_calc.append(energy_calc)
    names.append(name)

df = pandas.DataFrame({"x":energies_exp, "y":energies_calc, "name":names})

scatter = px.scatter(df, x="x", y="y", hover_name="name", symbol="name", color="name")
line = px.line(x=[-2.75, -0.75], y=[-2.75, -0.75])

fig = go.Figure(scatter.data + line.data)
fig.update_layout({"title": "Test of Carbonate Correction",
                   "xaxis_title":"Experimental dGf (eV/atom)",
                   "yaxis_title":"Calculated dGf (eV/atom)"})
fig.show()

In [121]:
from pymatgen.entries.computed_entries import GibbsComputedStructureEntry

temp = 1000  # Kelvin
gibbs_entries = GibbsComputedStructureEntry.from_entries(mp_ents, temp)